In [1]:
import os
import json
import gzip
import pandas as pd
from datetime import datetime

In [2]:
path_to_json = 'input/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json.gz')]
print(json_files)

['222qnhjnpu4tjmfbvsj2mvnsye.json.gz', 'yk5uuzph7m4l7fjzt6ymrci3fe.json.gz']


In [59]:
new_dict = {'vin' : [], 'timestamp' : [], 'status' : [], 'logs' : []}
for index, js in enumerate(json_files):
    with gzip.open(os.path.join(path_to_json, js), 'rb') as json_file:
        df = pd.read_json(json_file, lines=True)
        for f in df['Item']:
            new_dict['vin'].append(f['vin']['S'])
            new_dict['timestamp'].append(datetime.fromtimestamp(int(f['timestamp']['N'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
            new_dict['status'].append(json.loads(f['value']['S']))
            new_dict['logs'].append(json.loads(f['value']['S']))



In [295]:
df2 = pd.DataFrame.from_dict(new_dict)
df2

,vin,timestamp,status,logs
0,7039ef07e48e5d99cbd0435cf35d1eb7,2021-07-16 13:59:49,"{'clientTS': 1626424189364, 'status': 'ERROR',...","{'clientTS': 1626424189364, 'status': 'ERROR',..."
1,49fb42a88818ab4ff2fd79fc4b2b5937,2021-09-01 15:46:22,"{'clientTS': 1630491382330, 'status': 'ERROR',...","{'clientTS': 1630491382330, 'status': 'ERROR',..."
2,0d35e283af5fa8f665ea471572ca5dfb,2021-11-18 14:51:21,"{'clientTS': 1637227281690, 'status': 'ERROR',...","{'clientTS': 1637227281690, 'status': 'ERROR',..."
3,55d4159eabe7cdd350a45a9f275d457d,2021-07-09 11:52:04,"{'clientTS': 1625811724877, 'status': 'ERROR',...","{'clientTS': 1625811724877, 'status': 'ERROR',..."
4,abc25ab26bb95250d431abb60db79cba,2021-09-18 20:17:40,"{'clientTS': 1631976460633, 'status': 'ERROR',...","{'clientTS': 1631976460633, 'status': 'ERROR',..."
...,...,...,...,...
288630,c747664a6fe4d77f5d53fd44a46e7dd3,2021-10-14 19:18:19,"{'clientTS': 1634219299080, 'status': 'ERROR',...","{'clientTS': 1634219299080, 'status': 'ERROR',..."
288631,a3672ed6dd0cc3e5aec52bb848644a5a,2021-08-11 20:30:10,"{'clientTS': 1628694010985, 'status': 'ERROR',...","{'clientTS': 1628694010985, 'status': 'ERROR',..."
288632,85bb89817da3ed912b4fc8300f74e0c4,2021-10-13 15:48:05,"{'clientTS': 1634120285599, 'status': 'ERROR',...","{'clientTS': 1634120285599, 'status': 'ERROR',..."
288633,9a970333a24f1ca4e58b7ef9fcb484af,2021-11-12 16:17:07,"{'clientTS': 1636714027312, 'status': 'ERROR',...","{'clientTS': 1636714027312, 'status': 'ERROR',..."


In [296]:
df2['status'] = [f['status'] for f in df2['status']]
df2['logs'] = [f['logs'] for f in df2['logs']]

In [297]:
df2 = df2[df2['status'] == "OK"]

### Handling Log column to get required result

In [298]:

new_f = []
df2['marked'] = 0
for f in df2['logs']:
    f = f[1]
   
    if "ErrorCode::OK" in f['message']: #filter out rows having 'message': 'Commit is not required in Finalize'
        if 'from-map' in f['message']: #handle incorrect rows having from-key and to-key values in 'message' key itself.
            f['from-map'] = f['message'].split('from-map',1)[1]
            f['to-map'] = f['message'].split('to-map',1)[1]
        
        #df2.loc("420" in f['from-map'], "marked") = 1
    new_f.append(f)
df2['logs'] = new_f 
    

C:\Users\OMKARP~1\AppData\Local\Temp/ipykernel_6536/2395685577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['marked'] = 0
C:\Users\OMKARP~1\AppData\Local\Temp/ipykernel_6536/2395685577.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['logs'] = new_f


In [299]:
df2['logs']

5         {'updateMethod': 'OTA', 'message': 'Nds commit...
9         {'updateMethod': 'OTA', 'message': 'Nds commit...
14        {'updateMethod': 'OTA', 'message': 'Nds commit...
17        {'updateMethod': 'OTA', 'message': 'Nds commit...
22        {'updateMethod': 'OTA', 'message': 'Commit is ...
                                ...                        
288593    {'updateMethod': 'OTA', 'message': 'Nds commit...
288595    {'updateMethod': 'OTA', 'message': 'Nds commit...
288597    {'updateMethod': 'OTA', 'message': 'Nds commit...
288598    {'updateMethod': 'OTA', 'message': 'Nds commit...
288628    {'updateMethod': 'OTA', 'message': 'Nds commit...
Name: logs, Length: 45644, dtype: object

### Delete unnecessary exceptions from dataframe

In [300]:
df2.drop(df2[df2['logs'] == {'updateMethod': 'OTA', 'message': 'Commit is not required in Finalize', 'severity': 0, 'code': '0'}].index, inplace = True)
df2.drop(df2[df2['logs'] == {'updateMethod': 'OTA', 'message': 'Finishing commit on startup', 'severity': 0, 'code': '0'}].index, inplace = True)
df2.drop(df2[df2['logs'] == {'message': 'Finishing commit on startup', 'severity': 0, 'code': '0'}].index, inplace = True)
df2.drop(df2[df2['logs'] == {'message': 'Commit is not required in Finalize', 'severity': 0, 'code': '0'}].index, inplace = True)

e:\projects\test\nev\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [301]:
df2['logs'] 

5         {'updateMethod': 'OTA', 'message': 'Nds commit...
9         {'updateMethod': 'OTA', 'message': 'Nds commit...
14        {'updateMethod': 'OTA', 'message': 'Nds commit...
17        {'updateMethod': 'OTA', 'message': 'Nds commit...
28        {'updateMethod': 'OTA', 'message': 'Nds commit...
                                ...                        
288593    {'updateMethod': 'OTA', 'message': 'Nds commit...
288595    {'updateMethod': 'OTA', 'message': 'Nds commit...
288597    {'updateMethod': 'OTA', 'message': 'Nds commit...
288598    {'updateMethod': 'OTA', 'message': 'Nds commit...
288628    {'updateMethod': 'OTA', 'message': 'Nds commit...
Name: logs, Length: 44275, dtype: object

### Get records having 420 in "from-map" and 570 in "to-map" for logs column in dataframe

In [302]:
df2.loc[["420" in f['from-map'] and "570" in f['to-map'] for f in df2['logs']], "marked"] = 1

e:\projects\test\nev\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [303]:
df2[df2['marked'] == 1]

,vin,timestamp,status,logs,marked
674,9917e48c9cb7f86f86c035c1e2f025d7,2021-07-11 21:35:47,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
1107,6fefa67e6cb7dcbf4e25ad9cef43cb0b,2021-07-24 17:36:37,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
3321,8e1fb48db7f9b048d97417b3d3b2d3eb,2021-07-03 22:50:28,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
3566,a23adb8ffedb11b8ccf0ce4979d7a881,2021-07-09 14:31:01,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
4445,83654e82edbf19228a013c44158cb2e5,2021-07-22 20:47:29,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
...,...,...,...,...,...
281100,fc0816527e2c73ea73657f0134575fe5,2021-07-08 09:21:20,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
282585,e8c373b2608b7eb624ae91a65f517209,2021-07-10 18:04:09,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
286348,b9d92a7204003141ca7e9e7ffbef9c8e,2021-07-16 13:25:05,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1
286852,7e96c9d3377fdba82eca6f35613daa38,2021-07-03 00:21:56,OK,"{'updateMethod': 'OTA', 'message': 'Nds commit...",1


In [304]:
df2 = df2[df2['marked'] == 1]

In [281]:
df2.to_csv("test-output-data-with-correct-status.csv", mode='a', index= False)